In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-22"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11036,2024-05-22,Países Baixos Dbl,14:30,Groningen,Den Bosch,2.50,1.49,154.5,1.85,1.85,0.0,0.00,0.00,xdWupJyQ,0.400000,0.671141,0.540541,0.540541,0.071141,191.846,159.537447,0.831591,1.8,1.643168,0.912871,189.42,2.412,2.020351,0.837625,26.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,82,75,2.31,1.57,0.000,0.000,0.357984,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11037,2024-05-22,Israel Superliga,15:05,Maccabi Tel Aviv,Hapoel Tel-Aviv,1.55,2.39,173.5,1.87,1.79,0.0,0.00,0.00,lQI2IVLH,0.645161,0.418410,0.534759,0.558659,0.063571,110.382,15.122891,0.137005,2.4,1.341641,0.559017,119.32,1.234,0.197307,0.159892,55.0,100.794,6.196562,0.061477,1.8,1.643168,0.912871,96.76,1.254,0.077974,0.062181,14.0,76,82,1.57,1.18,102.254,124.828,0.301507,0.030912,NaN,0.56,0.112,4.910714,0.873277,1.0,0.126723,0.45,0.090,15.444444,0.711755,0.7,-0.011755
11038,2024-05-22,Israel Liga Leumit,14:00,Gilboa Galil,Maccabi Rehovot,1.50,2.47,149.5,1.85,1.85,0.0,0.00,0.00,SCsXqEsB,0.666667,0.404858,0.540541,0.540541,0.071525,154.590,37.426195,0.242100,3.0,0.000000,0.000000,95.36,1.868,0.392135,0.209922,35.0,133.200,20.953191,0.157306,1.8,1.643168,0.912871,147.50,1.854,0.427528,0.230598,16.0,64,59,1.49,2.50,140.612,131.390,0.345538,0.000000,NaN,3.96,0.792,0.631313,0.613025,0.7,0.086975,-1.19,-0.238,-6.176471,0.577703,0.6,0.022297
11039,2024-05-22,Itália Série A2,15:30,Chiusi,Luiss Roma,1.32,3.20,151.5,1.88,1.79,0.0,0.00,0.00,bJQH0HwA,0.757576,0.312500,0.531915,0.558659,0.070076,175.648,73.800888,0.420164,2.4,1.341641,0.559017,119.19,2.340,0.929731,0.397321,55.0,192.748,91.758164,0.476052,1.2,1.643168,1.369306,292.40,2.578,1.306912,0.506948,-26.0,87,85,1.37,3.44,191.036,284.678,0.588213,0.034681,NaN,3.34,0.668,0.479042,0.433526,0.5,0.066474,-5.00,-1.000,-2.200000,0.276075,0.2,-0.076075
11040,2024-05-22,Letônia Lbl,14:00,VEF Riga,Rigas Zelli,1.30,3.38,154.5,1.84,1.90,0.0,0.00,0.00,hIPweKTi,0.769231,0.295858,0.543478,0.526316,0.065089,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,255.976,100.336762,0.391977,1.8,1.643168,0.912871,317.80,3.286,1.206350,0.367118,21.0,64,70,1.19,4.54,0.000,246.828,0.628539,0.022688,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,3.93,0.786,3.027990,0.000000,0.0,0.000000
11041,2024-05-22,Lituânia Lkl,12:50,Rytas,BC Wolves,1.35,3.09,173.5,1.88,1.85,0.0,0.00,0.00,boXdIcU1,0.740741,0.323625,0.531915,0.540541,0.064365,113.992,9.028982,0.079207,2.4,1.341641,0.559017,105.60,1.144,0.147071,0.128559,77.0,157.728,85.558852,0.542446,2.4,1.341641,0.559017,116.28,1.758,0.815181,0.463698,19.0,96,76,1.10,1.53,121.962,130.374,0.554219,0.011374,NaN,0.07,0.014,25.000000,0.842511,0.8,-0.042511,-2.18,-0.436,-4.793578,0.610546,0.5,-0.110546
11042,2024-05-22,Nova Zelândia Nbl,04:30,Taranaki Airs,Bay Hawks,1.09,6.58,187.5,1.87,1.79,0.0,0.00,0.00,25v4vo9s,0.917431,0.151976,0.534759,0.558659,0.069407,144.462,39.341232,0.272329,2.4,1.341641,0.559017,109.08,1.478,0.490173,0.331646,92.0,330.766,225.235330,0.680951,1.2,1.643168,1.369306,184.86,3.328,2.191636,0.658545,-11.0,108,79,1.01,2.34,0.000,0.000,1.012260,0.030912,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11043,2024-05-22,Israel Superliga,12:45,Hapoel Jerusalem,Hap

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11047,13:00,Rússia Liga Vtb United,Unics Kazan,Lokomotiv Kuban,1.58,Back Home
11055,15:00,França Pro B,Orleans,Rouen,1.60,Back Home
11069,13:00,Bulgária Nbl,Chernomorets,Spartak Pleven,1.66,Back Home
11070,08:35,China Cba,Xinjiang,Liaoning,1.60,Back Home
